In [2]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

  Using cached sklearn-0.0.tar.gz (1.1 kB)
Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
ERROR: Could not install packages due to an EnvironmentError: [WinError 2] The system cannot find the file specified: 'c:\\python39\\Scripts\\f2py.exe' -> 'c:\\python39\\Scripts\\f2py.exe.deleteme'

You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


In [3]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

C:\Users\mille\Anaconda3\envs\PythonData\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


# Read the CSV and Perform Basic Data Cleaning

In [5]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [6]:
df = df[df.koi_disposition!="CANDIDATE"]

# Select your features (columns)

In [7]:
# Set features. This will also be used as your x values.
X = df.drop("koi_disposition", axis = 1)
X

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6983,0,1,0,0,21.513523,2.714000e-04,-2.714000e-04,132.335600,0.012200,-0.012200,...,-141,3.508,0.187,-0.153,3.318,0.665,-0.813,287.46786,37.966640,10.630
6986,0,0,0,1,8.589871,1.846000e-04,-1.846000e-04,132.016100,0.015700,-0.015700,...,-152,4.296,0.231,-0.189,1.088,0.313,-0.228,298.74921,46.973351,14.478
6987,0,1,1,0,0.527699,1.160000e-07,-1.160000e-07,131.705093,0.000170,-0.000170,...,-166,4.529,0.035,-0.196,0.903,0.237,-0.079,297.18875,47.093819,14.082
6989,0,0,1,0,0.681402,2.430000e-06,-2.430000e-06,132.181750,0.002850,-0.002850,...,-236,4.447,0.056,-0.224,1.041,0.341,-0.114,294.16489,47.176281,15.385


In [8]:
y = pd.get_dummies(df.koi_disposition)
y

,CONFIRMED,FALSE POSITIVE
0,1,0
1,0,1
2,0,1
3,1,0
4,1,0
...,...,...
6983,0,1
6986,0,1
6987,0,1
6989,0,1


In [9]:
print(X.shape, y.shape)

(5304, 40) (5304, 2)


In [10]:
# Create a Train Test Split

# Use `koi_disposition` for the y values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y.CONFIRMED)

In [11]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
1738,0,1,0,1,1.480324,3.370000e-06,-3.370000e-06,131.755300,0.002450,-0.002450,...,-180,4.342,0.153,-0.187,1.081,0.311,-0.207,282.01251,46.428200,13.660
6063,0,1,0,0,0.896271,3.710000e-07,-3.710000e-07,131.668052,0.000352,-0.000352,...,-166,4.493,0.081,-0.150,0.861,0.196,-0.105,295.00464,38.775478,15.261
2383,0,0,1,1,3.723561,6.500000e-06,-6.500000e-06,172.179770,0.001430,-0.001430,...,-212,4.373,0.062,-0.175,1.181,0.339,-0.145,290.41916,41.525372,14.930
2674,0,0,0,0,16.338881,1.880000e-05,-1.880000e-05,177.917586,0.000902,-0.000902,...,-53,3.645,0.060,-0.066,2.595,0.273,-0.198,290.32733,42.898281,13.781
5811,0,0,1,0,4.247489,5.960000e-05,-5.960000e-05,132.056200,0.012600,-0.012600,...,-192,4.524,0.042,-0.179,0.922,0.231,-0.093,298.38226,39.913521,15.146


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [12]:
# Scale your data
from numpy import asarray
from sklearn.preprocessing import MinMaxScaler

In [13]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Train the Model



In [15]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, y_train)
rf.score(X_test_scaled, y_test)

0.9864253393665159

In [23]:
print(rf.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 200, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [27]:
rf.get_params().keys()

dict_keys(['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [40]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
model = rf
param_grid = [
  {'n_estimators': [200, 100],
    'max_depth': [1, 5, 10],
  'criterion': ['gini', 'entropy']
  }]
grid = GridSearchCV(model, param_grid, verbose=3)

In [42]:
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] criterion=gini, max_depth=1, n_estimators=200 ...................
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV]  criterion=gini, max_depth=1, n_estimators=200, score=0.839, total=   0.5s
[CV] criterion=gini, max_depth=1, n_estimators=200 ...................
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[CV]  criterion=gini, max_depth=1, n_estimators=200, score=0.858, total=   0.4s
[CV] criterion=gini, max_depth=1, n_estimators=200 ...................
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.8s remaining:    0.0s
[CV]  criterion=gini, max_depth=1, n_estimators=200, score=0.845, total=   0.4s
[CV] criterion=gini, max_depth=1, n_estimators=200 ...................
[CV]  criterion=gini, max_depth=1, n_estimators=200, score=0.844, total=   0.4s
[CV] criterion=gini, max_depth=1, n_estimators=200 ...................
[CV]  criterion=gini, m

GridSearchCV(estimator=RandomForestClassifier(n_estimators=200),
             param_grid=[{'criterion': ['gini', 'entropy'],
                          'max_depth': [1, 5, 10],
                          'n_estimators': [200, 100]}],
             verbose=3)

In [43]:
print(grid.best_params_)
print(grid.best_score_)

{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 200}
0.9856704908188743


# Save the Model

In [44]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'Random Forest Model.sav'
joblib.dump(grid.best_score_, filename)

['Random Forest Model.sav']